#### Importing Important Libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

#### Exctracting Data

In [2]:
#restaurant name
def get_rest_name(soup):

    try:
        rest_name = soup.find(name='p',attrs={'class':'RestaurantNameAddress_name__2IaTv'})

        rest_name = rest_name.text.strip()

    except AttributeError:
        rest_name = ""

    return rest_name

#rating
def get_rest_rating(soup):
    try:
        rest_rating = soup.find(name='span',attrs={'class':'RestaurantRatings_avgRating__1TOWY'})
        rest_rating = rest_rating.text.strip()
    except AttributeError:
        rest_rating = ""
    return rest_rating

#price for 2
def get_rest_price2(soup):
    try:
        rest_price_2 = soup.find(name='ul', attrs={'class':'RestaurantTimeCost_wrapper__3YXF9'}).find_all(name='span')
        rest_price_2 = rest_price_2[-1].text.strip()
    except AttributeError:
        rest_price_2 = ""
    return rest_price_2

#Cuisine
def get_rest_cuis(soup):
    try:
        rest_loc = soup.find(name='p', attrs={'class':'RestaurantNameAddress_cuisines__mBHr2'})
        rest_loc = rest_loc.text.strip()
    except AttributeError:
        rest_loc = ""
    return rest_loc

#location
def get_rest_loc(soup):
    try:
        rest_loc = soup.find(name='p', attrs={'class':'RestaurantNameAddress_area__2P9ib'})
        rest_loc = rest_loc.text.strip()
    except AttributeError:
        rest_loc = ""
    return rest_loc

#Total Ratings
def get_total_ratings(soup):
    try:
        rest_loc = soup.find(name='span', attrs={'class':'RestaurantRatings_totalRatings__3d6Zc'})
        rest_loc = rest_loc.text.strip()
    except AttributeError:
        rest_loc = ""
    return rest_loc

In [3]:
if __name__ == '__main__':

    #Headers for request
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36','Accept-Language':'en-US,en;q=0.5'})

    #create request for webpage
    # webpage1 = requests.get(URL,headers=HEADERS)
    
    #URL - desired url
    url="Order food online from India's best food delivery service. Order from restaurants near you.html"
    with open(url, "r") as f:
        URL = f.read()

    #creating soup to store all html code
    soup = BeautifulSoup(URL,'html.parser')

    #get links to be scraped
    get_links1 = soup.find_all('a',attrs={'class':'_1j_Yo'})

    link_list = []
    for link in get_links1:
        link_list.append(link.get('href'))

    #create an empty dataset
    d={'restaurant_id':[],'restaurant_link':[],'name':[],'location':[],'rating':[],'total_ratings':[],'price_for_2':[],'cuisines':[]}

    #loop for extracting product details from each link
    for link in link_list:
        new_webpage = requests.get(link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, 'html.parser')

        #run all the functions to extract data per category  
        d['restaurant_id'].append(link.split('-')[-1])
        d['restaurant_link'].append(link)
        d['name'].append(get_rest_name(new_soup))
        d['rating'].append(get_rest_rating(new_soup))
        d['price_for_2'].append(get_rest_price2(new_soup))
        d['cuisines'].append(get_rest_cuis(new_soup))
        d['location'].append(get_rest_loc(new_soup))
        d['total_ratings'].append(get_total_ratings(new_soup))

#### Reading Dataframes

In [4]:
data = pd.DataFrame(d)
data

,restaurant_id,restaurant_link,name,location,rating,total_ratings,price_for_2,cusines
0,354531,https://www.swiggy.com/restaurants/burger-seig...,Burger Seigneur,Sobha Mall,4.4,1K+ ratings,₹1200 for two,"American, Continental"
1,428,https://www.swiggy.com/restaurants/biryani-pot...,Biryani Pot,Btm Layout,3.9,10K+ ratings,₹500 for two,"North Indian, Biryani"
2,5938,https://www.swiggy.com/restaurants/burger-king...,Burger King,Shivaji Nagar,4.1,10K+ ratings,₹350 for two,"Burgers, American"
3,312849,https://www.swiggy.com/restaurants/chopstick-a...,Chopstick,R T Nagar,4.0,1K+ ratings,₹350 for two,"Chinese, Thai"
4,30531,https://www.swiggy.com/restaurants/hotel-empir...,Hotel Empire,Hotel Empire International,4.2,10K+ ratings,₹450 for two,"North Indian, Kebabs"
...,...,...,...,...,...,...,...,...
760,641019,https://www.swiggy.com/restaurants/miami-waffl...,Miami Waffles & Jawbreakers,New Bel Road,NEW,,₹200 for two,"Waffle, Desserts"
761,520806,https://www.swiggy.com/restaurants/panda-momos...,Panda Momos,New Bel Road,3.8,20+ ratings,₹200 for two,"Asian, Tibetan"
762,692632,https://www.swiggy.com/restaurants/falafel-hou...,FALAFEL HOUSE,Kammanahalli/Kalyan Nagar,NEW,,₹300 for two,"Indian, Arabian"
763,669607,https://www.swiggy.com/restaurants/biristah-bi...,Biristah - Biryani & Rolls,Kammanahalli/Kalyan Nagar,4.2,100+ ratings,₹500 for two,"Biryani, Bengali"


#### Exporting Extracted Data as Excel/CSV File

In [5]:
data.to_csv('scraped_data.csv',index=False)

In [6]:
data.to_excel('scraped_data.xlsx',index=False)